# Data Preprocessing

In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_path='dataset'

# Data augmentation to improve model generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of augmented data
train_generator = train_datagen.flow_from_directory(
    #train_data_dir,
    data_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    data_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')


Found 1376 images belonging to 2 classes.
Found 1376 images belonging to 2 classes.


# CNN Model Training

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Build a simple CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

# Save the weights after training
model.save_weights('dataset.h5')

C:\Users\HP\AppData\Local\Temp\ipykernel_8052\4051009299.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
43/43 [==============================] - 27s 612ms/step - loss: 0.4730 - accuracy: 0.7602 - val_loss: 0.1386 - val_accuracy: 0.9499
Epoch 2/10
43/43 [==============================] - 27s 618ms/step - loss: 0.1179 - accuracy: 0.9615 - val_loss: 0.0657 - val_accuracy: 0.9818
Epoch 3/10
43/43 [==============================] - 28s 656ms/step - loss: 0.0828 - accuracy: 0.9702 - val_loss: 0.0506 - val_accuracy: 0.9840
Epoch 4/10
43/43 [==============================] - 40s 938ms/step - loss: 0.0627 - accuracy: 0.9767 - val_loss: 0.0285 - val_accuracy: 0.9927
Epoch 5/10
43/43 [==============================] - 35s 778ms/step - loss: 0.0403 - accuracy: 0.9862 - val_loss: 0.0197 - val_accuracy: 0.9956
Epoch 6/10
43/43 [==============================] - 26s 603ms/step - loss: 0.0487 - accuracy: 0.9826 - val_loss: 0.0334 - val_accuracy: 0.9884
Epoch 7/10
43/43 [==============================] - 27s 622ms/step - loss: 0.0465 - accuracy: 0.9833 - val_loss: 0.0537 - val_accuracy: 0.9746

# Mask Detection using Camera in Jupyter

In [7]:
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model.load_weights('weights.h5')

# Open camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    img = cv2.resize(frame, (150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255

    prediction = model.predict(img)
    mask_status = "Mask" if prediction[0][0] > 0.5 else "No Mask"

    cv2.putText(frame, mask_status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 24ms/step


...

1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 32ms/step


# Mask Detection Using Images

In [5]:
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model.load_weights('weights.h5')

# Specify the path to the image directory
image_directory = r'C:\Users\HP\Desktop\face_mask\test_image\withmask'

# Get a list of image files in the directory
image_files = [f for f in os.listdir(image_directory) if f.endswith(('.jpg', '.jpeg', '.png'))]

for image_file in image_files:
    # Read the image
    img_path = os.path.join(image_directory, image_file)
    img = cv2.imread(img_path)
    
    # Resize the image to match the input size of the model
    img = cv2.resize(img, (150, 150))
    
    # Preprocess the image for prediction
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255

    # Make predictions
    prediction = model.predict(img)
    mask_status = "Mask" if prediction[0][0] > 0.5 else "No Mask"

    # Display the result
    cv2.putText(img, mask_status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Mask Detection', img)
    cv2.waitKey(0)  # Wait for a key press before moving to the next image

cv2.destroyAllWindows()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\HP\\Desktop\\face_mask\\test_image\\withmask'